In [3]:
import pandas as pd
import numpy as np
from tmdbv3api import TMDb
import json
import requests
import bs4 as bs
import urllib.request
tmdb = TMDb()
with open("api-key.txt", mode = 'r') as f:
    tmdb.api_key = f.readline()

### Getting the data of movies released in 2020 so far from Wikipedia and TMDb

In [4]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2020"
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [5]:
tables = soup.find_all('table',class_='wikitable sortable')

In [6]:
len(tables)

4

In [7]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [8]:
data = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [9]:
data

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,3,The Grudge,Screen Gems / Stage 6 Films / Ghost House Pict...,Nicolas Pesce (director/screenplay); Andrea Ri...,[2]
1,JANUARY,10,Underwater,20th Century Fox / TSG Entertainment / Chernin...,"William Eubank (director); Brian Duffield, Ada...",[3]
2,JANUARY,10,Like a Boss,Paramount Pictures,"Miguel Arteta (director); Sam Pitman, Adam Col...",[4]
3,JANUARY,10,Inherit the Viper,Barry Films / Tycor International Film Company,Anthony Jerjen (director); Andrew Crabtree (sc...,[5]
4,JANUARY,10,The Sonata,Screen Media Films,Andrew Desmond (director/screenplay); Arthur M...,[6]
...,...,...,...,...,...,...
216,DECEMBER,25,News of the World,Universal Pictures,Paul Greengrass (director/screenplay); Luke Da...,[208]
217,DECEMBER,25,One Night in Miami,Amazon Studios,Regina King (director); Kemp Powers (screenpla...,[209]
218,DECEMBER,25,Promising Young Woman,Focus Features / FilmNation Entertainment,Emerald Fennell (director/screenplay); Carey M...,[210]
219,DECEMBER,25,Sylvie's Love,Amazon Studios,Eugene Ashe (director/screenplay); Tessa Thomp...,[211]


In [10]:
data = data[['Title','Cast and crew']]

In [13]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(title):
    '''Takes in the title of a movie, requests the genres from the TMDb api'''
    genres = []
    result = tmdb_movie.search(title)
    if len(result)==0:
        return ""
    movie_id = result[0].id
    response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}")
    data_json = response.json()
    if data_json['genres']:
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return " ".join(genres)
    else:
        np.NaN

In [15]:
print(tmdb_movie.search("Birds of Prey"))

[Birds of Prey (and the Fantabulous Emancipation of One Harley Quinn), Birds of Prey, Birds of Prey, Birds of Prey, Birds of Prey, Birds of Prey, Birds of Prey, Birds of Prey, Live Birds of Prey, The U.S. Ski Team's Greatest Day: 2004 Birds of Prey, Birds of Prey]


In [14]:
data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-14-1af8401fae03>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))


In [16]:
data.head()

,Title,Cast and crew,genres
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery


In [17]:
data_2020 = data[['Title','Cast and crew','genres']]

In [19]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [20]:
data_2020['director_name'] = data_2020['Cast and crew'].map(lambda x: get_director(x))

In [21]:
def get_actor(x, number = 1):
    if number == 2 and len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    if number == 3 and len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    return ((x.split("screenplay); ")[-1]).split(", ")[number - 1])

In [22]:
data_2020['actor_1_name'] = data_2020['Cast and crew'].map(lambda x: get_actor(x))
data_2020['actor_2_name'] = data_2020['Cast and crew'].map(lambda x: get_actor(x, 2))
data_2020['actor_3_name'] = data_2020['Cast and crew'].map(lambda x: get_actor(x, 3))

In [24]:
data_2020.head()

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,The Grudge,Nicolas Pesce (director/screenplay); Andrea Ri...,Horror Mystery,Nicolas Pesce,Andrea Riseborough,Demián Bichir,John Cho
1,Underwater,"William Eubank (director); Brian Duffield, Ada...",Action Horror Science Fiction Thriller,William Eubank,Kristen Stewart,Vincent Cassel,Jessica Henwick
2,Like a Boss,"Miguel Arteta (director); Sam Pitman, Adam Col...",Comedy,Miguel Arteta,Tiffany Haddish,Rose Byrne,Salma Hayek
3,Inherit the Viper,Anthony Jerjen (director); Andrew Crabtree (sc...,Drama Thriller Crime,Anthony Jerjen,Josh Hartnett,Margarita Levieva,Chandler Riggs
4,The Sonata,Andrew Desmond (director/screenplay); Arthur M...,Horror Thriller Mystery,Andrew Desmond,Freya Tingley,Simon Abkarian,Rutger Hauer


In [25]:
data_2020 = data_2020.rename(columns={'Title':'movie_title'})

In [26]:
new_2020 = data_2020.drop("Cast and crew", axis = 1)

In [27]:
new_2020.isnull().sum()

movie_title       0
genres            2
director_name     0
actor_1_name      0
actor_2_name      2
actor_3_name     21
dtype: int64

In [29]:
new_2020['actor_2_name'] = new_2020['actor_2_name'].replace(np.nan, 'unknown')
new_2020['actor_3_name'] = new_2020['actor_3_name'].replace(np.nan, 'unknown')

In [30]:
new_2020['comb'] = new_2020['actor_1_name'] + ' ' + new_2020['actor_2_name'] + ' '+ new_2020['actor_3_name'] + ' '+ new_2020['director_name'] +' ' + new_2020['genres']

In [32]:
new_2020.isna().sum()

movie_title      0
genres           2
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
comb             2
dtype: int64

In [35]:
new_2020 = new_2020.dropna(how='any')

In [36]:
new_2020['movie_title'] = new_2020['movie_title'].str.lower()

In [38]:
old_df = pd.read_csv('pre-2020_data.csv')

In [39]:
final_df = old_df.append(new_2020,ignore_index=True)

In [40]:
final_df.to_csv('final_data.csv',index=False)